In [3]:

import numpy as np
import pandas as pd

In [4]:
train_path="/home/u131168/mh_one_api/data/train.csv"
ftrain_path="/home/u131168/mh_one_api/model/formatted_traind.csv"

test_path="/home/u131168/mh_one_api/data/test.csv"
submission_path="/home/u131168/mh_one_api/data/submission.csv"

train_data=pd.read_csv(train_path)
ftrain_data=pd.read_csv(ftrain_path)

test_data=pd.read_csv(test_path)
submission_data=pd.read_csv(submission_path)


In [7]:
story=ftrain_data.iloc[0,0]
question=ftrain_data.iloc[0,1]
ftrain_data.iloc[:15]

ftrain_data.iloc[:,2].to_csv("/home/u131168/mh_one_api/data/train_split/train_output.csv",)


In [17]:

# !pip install bitsandbytes
# !pip install accelerate
# !pip install scipy

# %% pip install peft
from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer
import torch
model_path="/home/u131168/mh_one_api/model/ft_models/flan-t5-xl_peft_finetuned_model/checkpoint-4000"
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
custom_model = AutoPeftModelForSeq2SeqLM.from_pretrained(model_path, )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_path="google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_path)

ft5_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
import torch
prompt = "Today I believe we can finally"
prompt = [["Please answer to the following question."," Who is going to be the next john wick d'or?"], ]
# print(question,story)
prompt=[[story,question]]
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# sample up to 30 tokens
torch.manual_seed(0)  # doctest: +IGNORE_RESULT
outputs = custom_model.generate(input_ids=input_ids, do_sample=True, max_length=30)

pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True,clean_up_tokenization_spaces=True)
# ['Today I believe we can finally get rid of discrimination," said Rep. Mark Pocan (D-Wis.).\n\n"Just look at the']

In [49]:
p_count=50
prompts = ftrain_data.loc[:p_count,['input','instruction']].values.tolist()
print(prompts)

[['CHAPTER IV. \n\nNotwithstanding the earnest injunction that Maria had given to Mr. Delafield to continue where she left him, until her return, she expressed no surprise at not finding him in the room. The countenance of this young lady exhibited a droll mixture of playful mirth and sadness; she glanced her eyes once around the apartment, and perceiving it was occupied only by her friend, she said, laughing-- \n\n"Well, Charlotte, when is it to be? I think I retired in very good season." \n\n"Perhaps you did, Maria," returned the other, without raising her face from the reflecting attitude in which she stood--"I believe it is all very well." \n\n"Well! you little philosopher--I should think it was excellent--that--that is--if I were in your place. I suspected this from the moment you met." \n\n"What have you suspected, Maria?--what is it you imagine has occurred?" \n\n"What! why Seymour Delafield has been stammering--then he looked doleful--then he sighed--then he hemmed--then he sai

In [ ]:
res=[]

In [21]:
input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# sample up to 30 tokens
torch.manual_seed(0)  # doctest: +IGNORE_RESULT
outputs = custom_model.generate(input_ids=input_ids, do_sample=True, max_length=20)
pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [25]:
input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# sample up to 30 tokens
torch.manual_seed(0)  # doctest: +IGNORE_RESULT
outputs = ft5_model.generate(input_ids=input_ids, do_sample=True, max_length=20)
pred_ft5=tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [50]:
pred_custom_df=pd.DataFrame(pd.Series(pred_custom,name="pred_custom"))
pred_ft5_df=pd.DataFrame(pd.Series(pred_custom,name="pred_ft5"))
temp_df=pd.concat([ftrain_data[:p_count],pred_custom_df,pred_ft5_df],axis=1)
temp_df

,input,instruction,output,pred_custom,pred_ft5
0,CHAPTER IV. \n\nNotwithstanding the earnest in...,What emotions did she show?,mirth and sadness,playful mirth and sadness,playful mirth and sadness
1,"(CNN) -- Polk County, Florida, detectives arre...",What do police believe the motive to be?,robbery,robbery,robbery
2,"Malawi (, or ; or [maláwi]), officially the Re...",Is it a large country?,No,no,no
3,CHAPTER XXXII. \n\nMR. GILMORE'S SUCCESS. \n\n...,What reminded him of Bullhampton?,Everything,everything,everything
4,"Computer security, also known as cybersecurity...",Besides breaking or staling a computer what ot...,unknown,disruption or misdirection of the services,disruption or misdirection of the services
5,CHAPTER XV--THE END OF THE MEETING \n\nStephen...,With what?,with feminine intuition,"""Did he tell you that I had asked him to marry...","""Did he tell you that I had asked him to marry..."
6,BT Group plc (trading as BT) is a holding comp...,When did they sell?,in 1991 and 1993,1991 and 1993,1991 and 1993
7,"Steubenville, Ohio (CNN) -- One of the teens w...",Who said so?,"Steubenville,",Dennis McNamara,Dennis McNamara
8,"Gymnasts sprint down a runway, which is a maxi...",What is the maximum length of the runway for a...,25 meters,25 meters,25 meters
9,(CNN) -- All Blacks captain Richie McCaw is wa...,was it positive coverage?,no,i guess it changed a little,i guess it changed a little


In [25]:
prompts = test_data.loc[:500,['Story','Question']].values.tolist()
# print(prompts)
res=[]
input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# sample up to 30 tokens
torch.manual_seed(0)  # doctest: +IGNORE_RESULT
outputs = model.generate(input_ids, do_sample=True, max_length=20)
res=tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [26]:
# print(prompts)
print(res)

['Anxious', '125 million', '20', 'David Petraeus remained stoically aloof as a senior U', 'She wants to wait', '1923', 'European Union', '100-million-euro', 'yes', '__', 'the school', 'Wan Daxue', 'Shadow', "Israel's Chabad movement has set up a fund to provide for Moshe's", '1608', 'a goot feller', 'Mary has great firmness', 'watching out the window', 'Justin Ross Harris', 'Piedmont', 'reported her missing', 'savage beasts', 'Utah state police officer', '44', 'more than 150', 'yes', 'dance had broken up.', '24 March 1603', 'Indonesia', 'a roadside farm stand in upstate New York', 'Tuesday, Ukraine; Ukraine Prime Minister Mykola Azarov said in a late-', 'Ahmadinejad being re-elected', 'The territory consists of the main island, also named Tristan da Cunha, which', 'Becoming a mainstay on a show like "The X Factor"', 'yes', 'Paris', 'Aunt Tracy', 'could take months', 'US space exploration efforts have been led by NASA', 'New York Stock Exchange.', 'Miss Longestaffe', 'Duhan de Jandun', 

In [23]:
test_data

,Story,Question
0,"CHAPTER I \n\nON THE HOUSEBOAT \n\n""Say, Tom, ...",What was Sam's mood?
1,The Uralic languages (; sometimes called Urali...,By how many?
2,The University of Wisconsin–Madison (also know...,How many schools and colleges does it have?
3,(CNN) -- The unexpected resignation of David P...,What happened?
4,"CHAPTER THIRTY-TWO \n\nTENDER TROUBLES \n\n""Jo...",Why does Mrs. March not force her children's c...
...,...,...
28543,"CHAPTER III. \n\n""Nice customs curt'sy to grea...",Who was the Archbishop of Toledo that was of t...
28544,CHAPTER XI. \n\nIn the little dining-room of t...,Who lived there?
28545,"Chapter XXX. \n\n""I shall go on through all et...",is he fair?
28546,Chapter XXXIV \n\nViolence \n\n\n\nIt had been...,Where was he to stay until this trip?


In [27]:
temList=[0 for i in range(28548-len(res))]
submission_data=pd.Series(res+temList,name="Answer")
submission_data=pd.DataFrame(submission_data)
submission_data.to_csv("/home/u131168/mh_one_api/data/submissionflant5base.csv",index=False)
submission_data

,Answer
0,Anxious
1,125 million
2,20
3,David Petraeus remained stoically aloof as a s...
4,She wants to wait
...,...
28543,0
28544,0
28545,0
28546,0
